<a href="https://colab.research.google.com/github/Itsuki-Hamano123/auto_ml/blob/master/AdaNet/experiments/wine_cls.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [72]:
!pip install adanet

In [73]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive


In [74]:
%cd /gdrive/My Drive/機械学習練習/AutoML/Adanet/experiment
%ls

/gdrive/My Drive/機械学習練習/AutoML/Adanet/experiment
wine_cls.ipynb


# Wine分類モデルをAdaNetで構築

In [75]:
import pandas as pd

from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
import tensorflow as tf
import adanet

## データ読み込み

In [127]:
SEED = 2
LOG_DIR = 'logs/models'

wine_data = load_wine()

feature_names = wine_data.feature_names
target_names = wine_data.target_names

train_x, test_x, train_y, test_y = train_test_split(wine_data.data, wine_data.target,test_size=0.2, random_state=SEED)
train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size=0.2, random_state=SEED)
print('train_x:{}, train_y:{}, val_x:{}, val_y:{}, test_x:{}, test_y:{}'.format(train_x.shape, train_y.shape, val_x.shape, val_y.shape, test_x.shape, test_y.shape))

train_x:(113, 13), train_y:(113,), val_x:(29, 13), val_y:(29,), test_x:(36, 13), test_y:(36,)


In [77]:
def input_evaluation_set(x, feature_names, y):
    '''
    Parameters
    -----
    x : np.ndarray(x, d)
    features_name : list : string
    y : np.ndarray(d)

    Returns
    -----
    features : dict{'feature_name1':np.ndarray(d), 'feature_name2':np.ndarray(d),,,,,,,}
    labels : np.ndarray(n)

    INFO
    -----
      ----usage example-----
      features = {'SepalLength': np.array([6.4, 5.0]),
                  'SepalWidth':  np.array([2.8, 2.3]),
                  'PetalLength': np.array([5.6, 3.3]),
                  'PetalWidth':  np.array([2.2, 1.0])}
      labels = np.array([2, 1])
      return features, labels
      -----
    '''
    features = {}
    for key, f in zip(feature_names, x.T):
      features[key] = f
    labels = y
    return features, labels

In [99]:
train_features, train_labels = input_evaluation_set(x=train_x, feature_names=feature_names, y=train_y)
val_features, val_labels = input_evaluation_set(x=val_x, feature_names=feature_names, y=val_y)
test_feature, _ = input_evaluation_set(x=test_x, feature_names=feature_names, y=None)

In [79]:
def convert_feature_column(train_features):
  '''
  tfv2のEstimatorに渡せるfeature_columnを取得
  '''
  my_feature_columns = []
  for key in train_features.keys():
      my_feature_columns.append(tf.feature_column.numeric_column(key=key))
  return my_feature_columns

In [80]:
feature_columns = convert_feature_column(train_features)
feature_columns

[NumericColumn(key='alcohol', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='malic_acid', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='ash', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='alcalinity_of_ash', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='magnesium', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='total_phenols', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='flavanoids', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='nonflavanoid_phenols', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='proanthocyanins', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='color_intensity', shape=(1,), 

## AdaNet

In [136]:
# モデルの出力の定義
head= tf.estimator.MultiClassHead(n_classes=len(target_names))

run_config = tf.estimator.RunConfig(model_dir=LOG_DIR,
                                    tf_random_seed=SEED)

In [137]:
estimator = adanet.AutoEnsembleEstimator(head=head,
                                         ensemble_strategies=[adanet.ensemble.AllStrategy()], # 成長戦略を設定
                                         candidate_pool=lambda config:{
                                             'linear':
                                                tf.estimator.LinearEstimator(
                                                    head=head,
                                                    feature_columns=feature_columns,
                                                    config=run_config,
                                                    model_dir=LOG_DIR
                                                ),
                                              'dnn':
                                                tf.estimator.DNNEstimator(
                                                    head=head,
                                                    feature_columns=feature_columns,
                                                    config=run_config,
                                                    hidden_units=[10, 5],
                                                    model_dir=LOG_DIR
                                                )
                                         },
                                         config=run_config,
                                         max_iteration_steps=50,
                                         model_dir=LOG_DIR,
                                         )

INFO:tensorflow:Using config: {'_model_dir': 'logs/models', '_tf_random_seed': 2, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


INFO:tensorflow:Using config: {'_model_dir': 'logs/models', '_tf_random_seed': 2, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [138]:
def train_evalutaion_fn(features, labels, training=True, batch_size=256):
    '''
    トレーニング/検証データを提供する入力関数
    
    Parameters
    -----
    features : dict{'f_name':np.ndarray(d),,,,}
    labels : np.ndarray(d)

    Returns
    -----
    datasets : tf.data.Dataset
    '''
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

    if training:
      dataset = dataset.shuffle(1000).repeat()
    
    return dataset.batch(batch_size)

In [142]:
estimator.train(input_fn=lambda :train_evalutaion_fn(train_features, train_labels),
                steps=100)

INFO:tensorflow:Using config: {'_model_dir': 'logs/models', '_tf_random_seed': 2, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


INFO:tensorflow:Using config: {'_model_dir': 'logs/models', '_tf_random_seed': 2, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Using config: {'_model_dir': 'logs/models', '_tf_random_seed': 2, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


...

INFO:tensorflow:Saving checkpoints for 500 into logs/models/model.ckpt.


INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 500...


INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 500...


INFO:tensorflow:Loss for final step: 1.0491122.


INFO:tensorflow:Loss for final step: 1.0491122.


In [140]:
metrics = estimator.evaluate(input_fn=lambda :train_evalutaion_fn(val_features, val_labels, training=False))
metrics

INFO:tensorflow:Using config: {'_model_dir': 'logs/models', '_tf_random_seed': 2, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


INFO:tensorflow:Using config: {'_model_dir': 'logs/models', '_tf_random_seed': 2, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Using config: {'_model_dir': 'logs/models', '_tf_random_seed': 2, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


...

INFO:tensorflow:Saving dict for global step 150: accuracy = 0.41379312, average_loss = 8.773415, best_ensemble_index_0 = 0, best_ensemble_index_1 = 1, best_ensemble_index_2 = 1, global_step = 150, iteration = 2, loss = 8.773415


INFO:tensorflow:Saving dict for global step 150: accuracy = 0.41379312, average_loss = 8.773415, best_ensemble_index_0 = 0, best_ensemble_index_1 = 1, best_ensemble_index_2 = 1, global_step = 150, iteration = 2, loss = 8.773415


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 150: logs/models/model.ckpt-150


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 150: logs/models/model.ckpt-150


{'accuracy': 0.41379312,
 'architecture/adanet/ensembles': b'\nU\n\x13architecture/adanetB4\x08\x07\x12\x00B.| dnn | linear | dnn | linear | dnn | linear |J\x08\n\x06\n\x04text',
 'average_loss': 8.773415,
 'best_ensemble_index_0': 0,
 'best_ensemble_index_1': 1,
 'best_ensemble_index_2': 1,
 'global_step': 150,
 'iteration': 2,
 'loss': 8.773415}

In [123]:
def input_predict_fn(features, batch_size=256):
    '''
    予測用に入力データ整形
    '''
    # Convert the inputs to a Dataset without labels.
    return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)

In [124]:
predictions = estimator.predict(input_fn=lambda: input_predict_fn(test_feature))

INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp_nw_male', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp_nw_male', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [125]:
for result in predictions:
  class_id = pred_dict['class_ids'][0]
  probability = pred_dict['probabilities'][class_id]

  print('Prediction is "{cls_name}" ({score:.1f}%)'.format(cls_name=target_names[class_id], score=100 * probability))

INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp_nw_male', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp_nw_male', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp_nw_male', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


...

INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Done running local_init_op.


INFO:tensorflow:Done running local_init_op.


Prediction is "class_2" (100.0%)
Prediction is "class_2" (100.0%)
Prediction is "class_2" (100.0%)
Prediction is "class_2" (100.0%)
Prediction is "class_2" (100.0%)
Prediction is "class_2" (100.0%)
Prediction is "class_2" (100.0%)
Prediction is "class_2" (100.0%)
Prediction is "class_2" (100.0%)
Prediction is "class_2" (100.0%)
Prediction is "class_2" (100.0%)
Prediction is "class_2" (100.0%)
Prediction is "class_2" (100.0%)
Prediction is "class_2" (100.0%)
Prediction is "class_2" (100.0%)
Prediction is "class_2" (100.0%)
Prediction is "class_2" (100.0%)
Prediction is "class_2" (100.0%)
Prediction is "class_2" (100.0%)
Prediction is "class_2" (100.0%)
Prediction is "class_2" (100.0%)
Prediction is "class_2" (100.0%)
Prediction is "class_2" (100.0%)
Prediction is "class_2" (100.0%)
Prediction is "class_2" (100.0%)
Prediction is "class_2" (100.0%)
Prediction is "class_2" (100.0%)
Prediction is "class_2" (100.0%)
Prediction is "class_2" (100.0%)
Prediction is "class_2" (100.0%)
Prediction